In [ ]:
!git clone https://github.com/uonat/SS2023_DI-Lab_Precitaste.git

Cloning into 'SS2023_DI-Lab_Precitaste'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 169 (delta 34), reused 147 (delta 23), pack-reused 0
Receiving objects: 100% (169/169), 16.12 MiB | 16.40 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [ ]:
# Copy script to main directory to not dealing with directory issues
!cp '/content/SS2023_DI-Lab_Precitaste/scripts/download_rpc.sh' 'download_rpc.sh'

Upload kaggle.json file to runtime in contents folder before running below line.

To generate that json head to your account page and Account tab on that page: https://www.kaggle.com/settings/account. Then, click "Create New Token" button to download your own kaggle.json file. This is specific for user account. Upload the json file to the runtime and run below cell to move the file to the required folder.

In [ ]:
# For quicker experiment with only validation set, change
# unzip retail-product-checkout-dataset.zip line to
# unzip retail-product-checkout-dataset.zip val2019/*
# This will only unzip validation set
!bash ./download_rpc.sh

 62% 15.6G/25.3G [14:00<08:44, 19.9MB/s]
User cancelled operation
Unzipping dataset...
[retail-product-checkout-dataset.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of retail-product-checkout-dataset.zip or
        retail-product-checkout-dataset.zip.zip, and cannot find retail-product-checkout-dataset.zip.ZIP, period.


In [ ]:
%cd SS2023_DI-Lab_Precitaste

In [ ]:
!pip install . &> /dev/null

Setup for ViT-H model

In [ ]:
# This cell sets ViT-H model

import distutils.core
import sys,os
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'  &> /dev/null
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])} &> /dev/null
sys.path.insert(0, os.path.abspath('./detectron2'))

!wget 'https://dl.fbaipublicfiles.com/detectron2/ViTDet/LVIS/mask_rcnn_vitdet_h/332434656/model_final_866730.pkl'

from models.VitDet import load_model,Draw_pred_BB,Draw_original_seg

model_path ="/content/SS2023_DI-Lab_Precitaste/model_final_866730.pkl"
config_path = "/content/SS2023_DI-Lab_Precitaste/detectron2/projects/ViTDet/configs/LVIS/mask_rcnn_vitdet_h_100ep.py"
model = load_model(model_path,config_path) 

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from dataset.RPCDataset import RPCDataset

val_dataset_path = "/content/retail_product_checkout"
val_dataset = RPCDataset(val_dataset_path, "val")

In [ ]:
import cv2

def Draw_pred_BB(img, model_result):
    final_img = img.copy()
    for row_index in range(len(model_result[0]["instances"])):
        if model_result[0]["instances"].scores[row_index] > 0.4:
            x1,y1,x2,y2 = model_result[0]["instances"].pred_boxes[row_index].tensor.int().tolist()[0]
            cv2.rectangle(final_img, (x1*4, y1*4), (x2*4, y2*4), (43, 123, 255), 5)
    return final_img

def Draw_pred_BB_editted(img,p_b,p_s):
    final_img = img.copy()
    for row_index in range(len(p_b)):
        if p_s[row_index] > 0.5:
            x1,y1,x2,y2 = p_b[row_index]
            cv2.rectangle(final_img, (x1*4, y1*4), (x2*4, y2*4), (43, 123, 255), 5)
    return final_img

In [ ]:
from random import randint
import numpy as np
from PIL import Image
from torchvision import transforms
from utilities.non_maximum_suppression import nms
import matplotlib.pyplot as plt


rand_idx = randint(0, val_dataset.get_num_imgs()-1)

img_path = val_dataset.get_img_path_by_id(rand_idx)

pil_img = Image.open(img_path)

h,w = pil_img.size
img = transforms.PILToTensor()(pil_img.convert("RGB"))
resized_img = transforms.Resize((int(h/4), int(w/4)), antialias=None)(img)    
batch = [{'image':resized_img.to(device)}]
model.eval()
with torch.no_grad():
    model_result=model(batch)

In [ ]:
bbox_img = Draw_pred_BB(np.array(pil_img), model_result)
plt.figure()
plt.imshow(bbox_img)
plt.show()

In [ ]:
from utilities.non_maximum_suppression import run_nms 

picked_boxes, picked_score = run_nms(
    model_result[0]['instances'].pred_boxes.tensor.to(torch.int).tolist(),
    model_result[0]['instances'].scores.tolist(),
    np.asarray(img),
    0.5
)
nms_drawn_img = Draw_pred_BB_editted(np.array(pil_img), picked_boxes, picked_score)
plt.figure()
plt.imshow(nms_drawn_img)
plt.show()

In [ ]:
pred_txt_path = "/content/rpc-val-preds"
os.makedirs(pred_txt_path, exist_ok=True)

In [ ]:
import torch
import cv2
from utilities.non_maximum_suppression import nms 
from random import randint
import numpy as np
import torch
import cv2
import random 
from PIL import Image
from torchvision import transforms

# Using cpu is just for testing, code works with gpu too. 
# Might need some modifications on the code
model.to('cpu')
model.eval()

for i in range(val_dataset.get_num_imgs()):
  img_path = val_dataset.get_img_path_by_id(i)

  pil_img = Image.open(img_path)
  h,w = pil_img.size
  img = transforms.PILToTensor()(pil_img.convert("RGB"))
  resized_img = transforms.Resize((int(h/4), int(w/4)), antialias=None)(img)    
  batch = [{'image':resized_img.to(device)}]

  with torch.no_grad():
    model_result=model(batch)

  picked_boxes, picked_score = nms(
      model_result[0]['instances'].pred_boxes.tensor.to(torch.int).tolist(),
      model_result[0]['instances'].scores.tolist(),
      0.5)

  pred_img_name = val_dataset.img_info[i]['file_name']
  pred_txt_name = "".join(pred_img_name.split('.')[:-1]) + ".txt"
  pred_txt_path = os.path.join(pred_txt_path, pred_txt_name)

  with open(pred_txt_path, "w") as txtfile:
    for i,box in enumerate(picked_boxes):    
      conf = picked_score[i]
      
      if conf < 0.2:
          continue
      # Rescale bounding boxes to original dim
      x1, y1, x2, y2 = np.array(box) * 4
      w = x2 - x1
      h = y2 - y1
      # Above calculation of w, h can change from model to model
      # ViT-H outputs x1y1x2y2 but we need w and h
      txtfile.write("object {} {} {} {} {}\n".format(conf, x1, y1, w, h))